In [128]:
import pandas as pd
import os
import glob
try:
    code_dir
except NameError:
    code_dir = os.getcwd()
    base_dir = code_dir.replace("/codes_local", "")
    de_dir = base_dir + "/0_CD8_AIM+"
    de_dir_mm = base_dir + "/0_CD8_AIM+_mm"
    de_dir_mm_slt = base_dir + "/0_CD8_AIM+_mm_slt"

In [13]:
hs_mm_ref = 'https://raw.githubusercontent.com/TCellResearchTeam/T_cell_signature_Reference/master/Human_to_Mouse_geneNames.csv'
hs_mm_df = pd.read_csv(hs_mm_ref)
hs_to_mm_dict = {x:y for index, (x,y) in 
                 enumerate(zip(hs_mm_df['human_geneName'].tolist(),
                               hs_mm_df['mouse_geneName'].tolist()))}

## Create reverse direction comparison files

In [43]:
de_files = glob.glob("%s/*_differential.csv"%de_dir)

### Create reverse direction files
for de_file in de_files:
    de_df = pd.read_csv(de_file)
    
    # Create reverse direction file name
    de_file_dir = "/".join(de_file.split("/")[:-1])
    de_file_cp = de_file.split("/")[-1].replace("_differential.csv", "").split("_vs_")
    de_file_cp_rev = de_file_cp
    de_file_cp_rev.reverse()
    de_file_rev = de_file_dir + "/" + "_vs_".join(de_file_cp_rev) + "_differential.csv"
    
    # Reverse fc
    de_rev_df = de_df.copy()
    for i in de_rev_df.columns:
        if "logfc" in i:
            de_rev_df[i] = - de_rev_df[i]
    
    # Save
    de_rev_df.to_csv(de_file_rev, index=False)

## Human to mouse genename

In [58]:
de_files = glob.glob("%s/*_differential.csv"%de_dir)

In [132]:
# Convert human gene names to mouse gene names
for de_file in de_files:
    de_df = pd.read_csv(de_file, index_col=0)
    cvt_genes = list(set(de_df.index) & set(hs_to_mm_dict.keys())) # Find all convertable genes
    de_df = de_df.loc[cvt_genes]
    de_df.index = [hs_to_mm_dict[x] for x in de_df.index.tolist()] # Convert to mouse gene names
    
    # Save converted
    de_file_name = de_file.split("/")[-1]
    de_mm_file_name = de_dir_mm + "/" + de_file_name
    de_df.to_csv(de_mm_file_name)
    
    # Select top 500 for each category
    de_df_sig = de_df[de_df['t-test_overestim_var_padj'] <= 0.05]
    de_df_sig_neg = de_df_sig[de_df_sig['t-test_overestim_var_logfc'] < 0]
    de_df_sig_pos = de_df_sig[de_df_sig['t-test_overestim_var_logfc'] > 0]
    de_df_sig_neg_slt = de_df_sig_neg.sort_values('t-test_overestim_var_logfc') # [:500]
    de_df_sig_pos_slt = de_df_sig_pos.sort_values('t-test_overestim_var_logfc', ascending=False) # [:500]
    de_df_sig_slt = de_df_sig_neg_slt.append(de_df_sig_pos_slt)
    de_mm_slt_file_name = de_dir_mm_slt + "/" + de_file_name
    de_df_sig_slt.to_csv(de_mm_slt_file_name)

In [133]:
de_df_sig_slt

,moderate,none,sev,wilcoxon_logfc,wilcoxon_padj,wilcoxon_score,t-test_logfc,t-test_padj,t-test_score,t-test_overestim_var_logfc,t-test_overestim_var_padj,t-test_overestim_var_score
Ocrl,0.000000,0.000000,0.008830,-23.080339,0.998941,-0.231191,-23.080339,0.182637,-2.009654,-23.080339,0.001864,-3.653888
Mast1,0.000000,0.000000,0.008830,-23.080339,0.998941,-0.231191,-23.080339,0.182637,-2.009654,-23.080339,0.001864,-3.653888
Amn,0.000000,0.000000,0.007914,-22.921637,0.998941,-0.145495,-22.921637,0.286123,-1.692704,-22.921637,0.010354,-3.077620
Dscaml1,0.000000,0.000000,0.006622,-22.663708,0.998941,-0.145495,-22.663708,0.268813,-1.737611,-22.663708,0.008238,-3.159268
Rapgef3,0.000000,0.000000,0.006622,-22.663708,0.998941,-0.145495,-22.663708,0.268813,-1.737611,-22.663708,0.008238,-3.159268
...,...,...,...,...,...,...,...,...,...,...,...,...
Anp32b,1.213816,0.232428,1.147684,0.137706,0.464638,1.984797,0.137706,0.247751,1.806758,0.137706,0.037456,2.562424
Uba52,2.625378,0.954639,2.541082,0.131553,0.028726,3.341344,0.131553,0.016624,3.195203,0.131553,0.000062,4.580524
Tomm7,1.572218,0.526701,1.505144,0.123227,0.484427,1.956671,0.123227,0.201966,1.939437,0.123227,0.019261,2.842378
Oaz1,1.820589,0.639988,1.751902,0.119051,0.655388,1.699911,0.119051,0.255569,1.785850,0.119051,0.034898,2.592048
